# Advanced Lane Finding Project

---

**Author:** Sergey Morozov

---

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
import logging

# set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s [%(levelname)s] %(message)s')

# constants
CHESSBOARD_IMAGE_DIR = "chessboard_images/"

# all lane finding code is in advanced_lane_finding.py
from advanced_lane_finding import *

## Compute Camera Calibration Matrix and Distortion Coefficients

In [ ]:
lane_finder = AdvancedLaneFinder(chessboard_image_dir=CHESSBOARD_IMAGE_DIR)